In [2]:
from bs4 import BeautifulSoup
import lxml
import requests
import re
import pandas

## Idea

Could we use this data and then the data from wikipedia/imdb (ratings) to see if the episodes that Karen's is on are better than the typical episode?

In [11]:
url = 'https://theoffice.fandom.com/wiki/The_Convict'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')

In [55]:
# finding the next link
table = soup.find('section', class_ = 'pi-item pi-group pi-border-color')
# find the link to the next episode
next_ep = table.find_all('div', class_='pi-data-value pi-font')[-1].find(href=True)['href']

# find's a lot of good information
ele = soup.find_all('div', class_ = 'pi-data-value pi-font') 

# the recurring cast
page_uls = soup.find_all('ul')

ele[7].text

"""There's no good way to find the uls because we have no id so what I do is I go through all the ul's and see which
one has a previous element that has the text of 'recurring cast.' 
"""
for ul in page_uls:
    if ul.find_previous().text == 'Recurring cast':
        recur_cast_box = ul
        
page_uls[-12].find_previous().text
#dir(soup)

'Recurring cast'

In [65]:
# finding the second link which is the character's name

recurring_cast = []

for char in recur_cast_box.find_all("a")[1::2]: # start with the character name and skip each actor name
    recurring_cast.append(char.text)
recurring_cast

['Andy Bernard',
 'Karen Filippelli',
 'Creed Bratton',
 'Martin Nash',
 'Hannah Smoterich-Barr']

In [13]:
next_ep

'/wiki/A_Benihana_Christmas'

In [75]:
base = 'https://theoffice.fandom.com'
url = 'https://theoffice.fandom.com/wiki/Pilot'
links = []
count = 0

titles = []
seasons = []
dates = []
writers = []
directors = []

ep_recurring_cast = []

while True:
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    # finding data
    ele = soup.find_all('div', class_ = 'pi-data-value pi-font')
    
    season = ele[0].text
    seasons.append(season)
    
    date = ele[3].text
    dates.append(date)
    
    writer = ele[4].text
    writers.append(writer)
    
    director = ele[5].text
    directors.append(director)
    
    title = ele[7].text
    titles.append(title)
    
    # find the recurring cast
    
    recurring_cast = [] # this will 
    
    page_uls = soup.find_all('ul') # we find the uls which is the box that has the recurring cast
    
    for ul in page_uls:
        if ul.find_previous().text == 'Recurring cast': # find the ul that has a element whose text is 'Recurring Cast'
            recur_cast_box = ul

            for char in recur_cast_box.find_all("a")[1::2]: # start with the character name and skip each actor name
                recurring_cast.append(char.text) 
    # add the sub-list
    recurring_cast = ";".join(recurring_cast)
    ep_recurring_cast.append(recurring_cast)
        
    
    # finding the the next episode
    table = soup.find('section', class_ = 'pi-item pi-group pi-border-color')  # table that has a box
    try:
        next_ep = table.find_all('div', class_='pi-data-value pi-font')[-1].find(href=True)['href']  # find the last element in the box, and in that, find a link
    except TypeError: # when it hits Finale, it won't find a the href and it'll return an empty list which can't be subscripted and that'll return a type error
        break
    
    url = base + next_ep
    links.append(url)
    
    
    count += 1
    if count == 5:
        break

In [76]:
ep_recurring_cast

['Jan Levinson;Roy Anderson;Stanley Hudson;Kevin Malone;Angela Martin;Oscar Martinez;Phyllis Vance;Creed Bratton',
 'Stanley Hudson;Kevin Malone;Meredith Palmer;Kelly Kapoor;Toby Flenderson;Angela Martin;Oscar Martinez;Phyllis Lapin;Creed Bratton;Devon White',
 'Jan Levinson;Stanley Hudson;Kevin Malone;Meredith Palmer;Kelly Kapoor;Angela Martin;Toby Flenderson;Oscar Martinez;Phyllis Vance;Creed Bratton;Devon Abner',
 'Roy Anderson;Stanley Hudson;Kevin Malone;Meredith Palmer;Angela Martin;Toby Flenderson;Oscar Martinez;Phyllis Vance;Darryl Philbin;Creed Bratton;Devon White',
 'Darryl Philbin;Roy Anderson;Lonnie Collins;Jerry DiCanio;Madge Madsen;Stanley Hudson;Kevin Malone;Meredith Palmer;Angela Martin;Toby Flenderson;Oscar Martinez;Phyllis Vance;Jan Levinson;Philip;Creed Bratton;Devon White']

In [77]:
import pandas as pd

In [78]:
d = pd.DataFrame({"season":seasons,
                  "name":titles,
                  "dates":dates,
                  "writer":writers,
                  "directors":directors,
                  "recurring_cast":ep_recurring_cast})

In [79]:
d

,season,name,dates,writer,directors,recurring_cast
0,Season 1,Pilot,"March 24, 2005",Ricky Gervais & Stephen Merchant and Greg Daniels,Ken Kwapis,Jan Levinson;Roy Anderson;Stanley Hudson;Kevin...
1,1,Diversity Day,"March 29, 2005",B.J. Novak,Ken Kwapis,Stanley Hudson;Kevin Malone;Meredith Palmer;Ke...
2,1,Health Care,"April 5, 2005",Paul Lieberstein,Ken Whittingham,Jan Levinson;Stanley Hudson;Kevin Malone;Mered...
3,1,The Alliance,"April 12, 2005",Michael Schur,Bryan Gordon,Roy Anderson;Stanley Hudson;Kevin Malone;Mered...
4,1,Basketball,"April 19, 2005",Greg Daniels,Greg Daniels,Darryl Philbin;Roy Anderson;Lonnie Collins;Jer...


In [170]:
d.to_csv("the_office_data.csv")